In [ ]:
import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "tx generator - setup"

In [ ]:
from case_code.code_downloader import CodeDownloader, get_metadata
from case_code.code_loader import CodeLoader
from case_code.code_transformer import transform

In [ ]:
from datetime import datetime

skip_download = True

try:
    metadata = get_metadata()
    dt_object = datetime.fromtimestamp(metadata["last_updated"])
    readable_date = dt_object.strftime("%Y-%m-%d %H:%M:%S")
    # Ask user if they want to download the latest files
    response = input(
        f"Last updated was {readable_date}. Do you want to download the latest files? (y/n)"
    )

    skip_download = response.lower() == "n"
except:
    # If there is no metadata, download the latest files
    print("No metadata found. Downloading latest files.")

In [ ]:
if skip_download:
    downloader = CodeDownloader()
    downloaded_count = downloader.download()

In [ ]:
metadata = get_metadata()
print(f"Total files: {metadata['total_files']}")
print(f"Total cases: {metadata['total_cases']}")

In [ ]:
loader = CodeLoader()

transformed_stats = await transform(loader=loader)

In [ ]:
from case_code.code_transformer import get_transformed_stats, save_transformed_stats

get_transformed_stats()